In [586]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import numpy as np


# 输入所有agent的obs，输出所有agent的动作概率分布
class G2ANet(nn.Module):
    def __init__(self, input_shape, args):
        super(G2ANet, self).__init__()

        # Encoding
        self.encoding = nn.Linear(input_shape, args.rnn_hidden_dim)  # 对所有agent的obs解码
        self.h = nn.GRUCell(args.rnn_hidden_dim, args.rnn_hidden_dim)  # 每个agent根据自己的obs编码得到hidden_state，用于记忆之前的obs

        # Hard
        # GRU输入[[h_i,h_1],[h_i,h_2],...[h_i,h_n]]与[0,...,0]，输出[[h_1],[h_2],...,[h_n]]与[h_n]， h_j表示了agent j与agent i的关系
        # 输入的iputs维度为(n_agents - 1, batch_size * n_agents, rnn_hidden_dim * 2)，
        # 即对于batch_size条数据，输入每个agent与其他n_agents - 1个agents的hidden_state的连接
        self.hard_bi_GRU = nn.GRU(args.rnn_hidden_dim * 2, args.rnn_hidden_dim, bidirectional=True)
        # 对h_j进行分析，得到agent j对于agent i的权重，输出两维，经过gumble_softmax后取其中一维即可，如果是0则不考虑agent j，如果是1则考虑
        self.hard_encoding = nn.Linear(args.rnn_hidden_dim * 2, 2)  # 乘2因为是双向GRU，hidden_state维度为2 * hidden_dim

        # Soft
        self.q = nn.Linear(args.rnn_hidden_dim, args.attention_dim, bias=False)
        self.k = nn.Linear(args.rnn_hidden_dim, args.attention_dim, bias=False)
        self.v = nn.Linear(args.rnn_hidden_dim, args.attention_dim)

        # Decoding 输入自己的h_i与x_i，输出自己动作的概率分布
        self.decoding = nn.Linear(args.rnn_hidden_dim + args.attention_dim, args.n_actions)
        self.args = args
        self.input_shape = input_shape
        self.softmax=f.softmax

    def forward(self, obs, hidden_state):
        size = obs.shape[0]  # batch_size * n_agents
        # 先对obs编码
        obs_encoding = f.relu(self.encoding(obs))
        h_in = hidden_state.reshape(-1, self.args.rnn_hidden_dim)

        # 经过自己的GRU得到h
        h_out = self.h(obs_encoding, h_in)  # (batch_size * n_agents, args.rnn_hidden_dim)

        # Hard Attention，GRU和GRUCell不同，输入的维度是(序列长度,batch_size, dim)
        if self.args.hard:
            # Hard Attention前的准备
            h = h_out.reshape(-1, self.args.n_agents, self.args.rnn_hidden_dim)  # 把h转化出n_agents维度，(batch_size, n_agents, rnn_hidden_dim)
            input_hard = []
            for i in range(self.args.n_agents):
                h_i = h[:, i]  # (batch_size, rnn_hidden_dim)
                h_hard_i = []
                for j in range(self.args.n_agents):  # 对于agent i，把自己的h_i与其他agent的h分别拼接
                    if j != i:
                        h_hard_i.append(torch.cat([h_i, h[:, j]], dim=-1))
                # j 循环结束之后，h_hard_i是一个list里面装着n_agents - 1个维度为(batch_size, rnn_hidden_dim * 2)的tensor
                h_hard_i = torch.stack(h_hard_i, dim=0)
                input_hard.append(h_hard_i)
            # i循环结束之后，input_hard是一个list里面装着n_agents个维度为(n_agents - 1, batch_size, rnn_hidden_dim * 2)的tensor
            input_hard = torch.stack(input_hard, dim=-2)
            # 最终得到维度(n_agents - 1, batch_size * n_agents, rnn_hidden_dim * 2)，可以输入了
            input_hard = input_hard.view(self.args.n_agents - 1, -1, self.args.rnn_hidden_dim * 2)

            h_hard = torch.zeros((2 * 1, size, self.args.rnn_hidden_dim))  # 因为是双向GRU，每个GRU只有一层，所以第一维是2 * 1
            if self.args.cuda:
                h_hard = h_hard.cuda()
            h_hard, _ = self.hard_bi_GRU(input_hard, h_hard)  # (n_agents - 1,batch_size * n_agents,rnn_hidden_dim * 2)
            h_hard = h_hard.permute(1, 0, 2)  # (batch_size * n_agents, n_agents - 1, rnn_hidden_dim * 2)
            h_hard = h_hard.reshape(-1, self.args.rnn_hidden_dim * 2)  # (batch_size * n_agents * (n_agents - 1), rnn_hidden_dim * 2)

            # 得到hard权重, (n_agents, batch_size, 1,  n_agents - 1)，多出一个维度，下面加权求和的时候要用
            hard_weights = self.hard_encoding(h_hard)
            hard_weights = f.gumbel_softmax(hard_weights, tau=0.01)
            # print(hard_weights)
            hard_weights = hard_weights[:, 1].view(-1, self.args.n_agents, 1, self.args.n_agents - 1)
            hard_weights = hard_weights.permute(1, 0, 2, 3)

        else:
            hard_weights = torch.ones((self.args.n_agents, size // self.args.n_agents, 1, self.args.n_agents - 1))
            if self.args.cuda:
                hard_weights = hard_weights.cuda()

        # Soft Attention
        q = self.q(h_out).reshape(-1, self.args.n_agents, self.args.attention_dim)  # (batch_size, n_agents, args.attention_dim)
        k = self.k(h_out).reshape(-1, self.args.n_agents, self.args.attention_dim)  # (batch_size, n_agents, args.attention_dim)
        v = f.relu(self.v(h_out)).reshape(-1, self.args.n_agents, self.args.attention_dim)  # (batch_size, n_agents, args.attention_dim)
        x = []
        for i in range(self.args.n_agents):
            q_i = q[:, i].view(-1, 1, self.args.attention_dim)  # agent i的q，(batch_size, 1, args.attention_dim)
            k_i = [k[:, j] for j in range(self.args.n_agents) if j != i]  # 对于agent i来说，其他agent的k
            v_i = [v[:, j] for j in range(self.args.n_agents) if j != i]  # 对于agent i来说，其他agent的v

            k_i = torch.stack(k_i, dim=0)  # (n_agents - 1, batch_size, args.attention_dim)
            k_i = k_i.permute(1, 2, 0)  # 交换三个维度，变成(batch_size, args.attention_dim， n_agents - 1)
            v_i = torch.stack(v_i, dim=0)
            v_i = v_i.permute(1, 2, 0)

            # (batch_size, 1, attention_dim) * (batch_size, attention_dim，n_agents - 1) = (batch_size, 1，n_agents - 1)
            score = torch.matmul(q_i, k_i)

            # 归一化
            scaled_score = score / np.sqrt(self.args.attention_dim)

            # softmax得到权重
            soft_weight = f.softmax(scaled_score, dim=-1)  # (batch_size，1, n_agents - 1)

            # 加权求和，注意三个矩阵的最后一维是n_agents - 1维度，得到(batch_size, args.attention_dim)
            x_i = (v_i * soft_weight * hard_weights[i]).sum(dim=-1)
            x.append(x_i)

        # 合并每个agent的h与x
        x = torch.stack(x, dim=1).reshape(-1, self.args.attention_dim)  # (batch_size * n_agents, args.attention_dim)
        final_input = torch.cat([h_out, x], dim=-1)
        output = self.softmax(1*self.decoding(final_input))
        return output

In [587]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import argparse
from copy import deepcopy
import torch
import gym
import time
import torch
import torch.nn as nn	# 各种层类型的实现
import torch.nn.functional as F	# 各中层函数的实现，与层类型对应，如：卷积函数、池化函数、归一化函数等等
import torch.optim as optim	# 实现各种优化算法的包
import numpy as np

import argparse
import sys


import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt
from grid_data import GridData

from compute_mpe import CircuitMPE
import sys
sys.path.insert(0,'/root/gurobi903/linux64/lib/python3.6_utf32/gurobipy')
import gurobipy
cmpe = CircuitMPE('/root/PySDD/notebooks/yahoo.vtree', '/root/PySDD/notebooks/yahoo.sdd')
class para:
    def __init__(self):
        self.batch_size=16
        self.wmc=10
        self.solDim=24
        self.card=10
        self.test_batch_size=1000
        self.epochs=10
        self.lr=0.1
        self.momentum=0.5
        self.no_cuda='store_true'
        self.seed=1
        self.log_interval=10
        self.hidden_dim=10
args=para()
legalList={0: [13, 18],
 1: [17, 20, 22],
 2: [4, 5, 10, 13, 14, 16, 17, 18, 20, 22],
 3: [5, 9, 11, 12, 15],
 4: [2, 10, 13, 14, 16, 17, 18, 20, 22],
 5: [2, 3, 6, 9, 10, 11, 12, 14, 16],
 6: [5, 15],
 7: [],
 8: [],
 9: [3, 5, 11, 12, 15],
 10: [2, 4, 5, 14, 16, 22],
 11: [3, 5, 9, 12, 15],
 12: [3, 5, 9, 11, 15],
 13: [0, 2, 4, 14, 18, 22],
 14: [2, 4, 5, 10, 13, 16, 17, 18, 20, 22],
 15: [3, 6, 9, 11, 12, 17, 19, 20],
 16: [2, 4, 5, 10, 14, 22],
 17: [1, 2, 4, 14, 15, 19, 20, 22],
 18: [0, 2, 4, 13, 14, 22],
 19: [15, 17, 20],
 20: [1, 2, 4, 14, 15, 17, 19, 22],
 21: [],
 22: [1, 2, 4, 10, 13, 14, 16, 17, 18, 20],
 23: []}
def checkFea(x):
    cnt=0
    cntT=0
    for i in range(len(legalList.keys())):
        for j in legalList[i]:
            cntT+=1
            if x[j]+x[i]>1:
                cnt+=1
    return cnt/cntT/2+abs(sum(x)-args.card)/args.card/2
a = 10*(np.load('rateListUsersYahoo.npy')[0]-np.load('rateListUsersYahoo.npy')[0].min())#np.random.random(24) #  np.load('rateListUsersYahoo.npy')[0]
a /= np.linalg.norm(a, ord=2)
a=torch.from_numpy(a).float()
Q=np.load('Q24.npy')#Q=np.random.random([n_features,n_features])+100*np.diag(np.ones(n_features))
#h = lambda x:   100*a.dot(Q).dot(a)   #min(100*(np.dot(a, x)**(5)),10000)
#100*a.dot(Q).dot(a)
def constraint(x):
    yu=torch.unbind(x)
    xu = torch.unbind(torch.from_numpy(np.array([0.0]*args.solDim).astype(np.float32)))
    wmc = cmpe.get_torch_ac([[1.0 - ny,ny] for ny in yu + xu]) 
    return  50000*torch.log(torch.mean(wmc)).float()
hwithoutconstraints = lambda x:10*(x.dot(a))**1/(np.sqrt(args.card))**1 #100*(x.dot(a))**2+100*x.dot(Q).dot(x) # 、100*x.dot(Q).dot(x)

In [541]:
import torch.optim as optim
from torch.autograd import Variable
class g2apara:
    def __init__(self):
        self.rnn_hidden_dim=64
        self.n_agents=args.solDim
        self.attention_dim=16
        self.cuda=False
        self.n_actions=2
        self.hard=False
args3=g2apara()
g2a=G2ANet(1,args3)
def weights_init(mod):
    """设计初始化函数"""
    classname=mod.__class__.__name__
    # 返回传入的module类型
    print(classname)
    if classname.find('Conv')!= -1:    #这里的Conv和BatchNnorm是torc.nn里的形式
        mod.weight.data.normal_(-0.1,0.2)
    elif classname.find('BatchNorm')!= -1:
        mod.weight.data.normal_(1.0,0.2) #bn层里初始化γ，服从（1，0.02）的正态分布
        mod.bias.data.fill_(0)  #bn层里初始化β，默认为0

g2a.apply(weights_init)
optimizer = optim.SGD(g2a.parameters(), lr=1e-3, momentum=1e-1)
g2a.train()
hids=torch.from_numpy(np.random.random([args.solDim,64])).float()
rewards=[]
gamma=0.99
net = torch.nn.Sequential(
        torch.nn.Linear(args.solDim, 200),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(200, 100),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(100, 50),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(50, 1),
    )
optimizer1 = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

for t in range(1000):
    if t<100:
        tmp=[1]*args.card+[0]*(args.solDim-args.card)
        np.random.shuffle(tmp)
        res=torch.tensor(tmp)
    else:
        action=g2a.forward(torch.tensor([1.]*args.solDim).view(args.solDim,1),hids)
        probs =[i for i in action]
        res=[probs[i].multinomial(1).data[0] if probs[i][0] not in [0.,1.] else probs[i][1] for i in range(args.solDim)]
        while (sum(res)-args.card)**2<=25 and sum(res)!=args.card:
            res=[probs[i].multinomial(1).data[0] if probs[i][0] not in [0.,1.] else probs[i][1] for i in range(args.solDim)]
        prob = [probs[i][res[i]] if probs[i][0] not in [0.,1.] else 1.  for i in range(args.solDim)]
        log_prob = [prob[i].log()  if probs[i][0] not in [0.,1.] else 0. for i in range(args.solDim)]
        entropy = sum([-(probs[i]*probs[i].log()).sum() if probs[i][0] not in [0.,1.] else 0. for i in range(args.solDim)])
        log_prob=sum(log_prob)
        optimizer.zero_grad()     #等效netD.zero_grad()
        reward=hwithoutconstraints(torch.tensor(res).float())-10*checkFea(res)-1*(sum(res)-args.card)**2#-constraint(torch.cat(tuple([i[1].view(1) for i in action]),0))/10000+((sum(([i[1] for i in action]))-args.card)**2)+(sum(([float(i[1]>0.5) for i in action]))-args.card)**2+1*entropy
        R = torch.zeros(1)
        R = gamma * R + reward
        loss =  -log_prob*Variable(R) - 5*entropy
        #loss=sum((sum(action)**2))/10#,torch.tensor(sum((1000*sum(action)**4)),requires_grad=True)
        #(hwithoutconstraints(torch.tensor([float(i[1]>0.5) for i in action]))-10*checkFea(torch.tensor([float(i[1]>0.5) for i in action])))
        loss.backward()#(retain_graph=True)  
        optimizer.step()   # update parameters
    prediction=net(torch.tensor(res).detach().float())
    loss1 = loss_func(prediction, hwithoutconstraints(torch.tensor(res).float()))
    print(t,hwithoutconstraints(torch.tensor(res).float())-10*checkFea(res),sum(res),loss1.item())
    optimizer1.zero_grad()   # clear gradients for next train
    loss1.backward()         # backpropagation, compute gradients
    optimizer1.step()        # apply gradients

Linear
GRUCell
GRU
Linear
Linear
Linear
Linear
Linear
G2ANet
0 tensor(4.1487) tensor(10) 30.95217514038086
1 tensor(5.8567) tensor(10) 39.3326416015625
2 tensor(5.9616) tensor(10) 35.77177429199219
3 tensor(3.8220) tensor(10) 5.1829915046691895
4 tensor(4.8644) tensor(10) 3.453937292098999
5 tensor(6.2912) tensor(10) 2.208454132080078
6 tensor(5.9342) tensor(10) 6.463838577270508
7 tensor(3.9205) tensor(10) 19.031770706176758
8 tensor(5.5349) tensor(10) 4.3416314125061035
9 tensor(5.5920) tensor(10) 0.6929411292076111
10 tensor(5.3599) tensor(10) 1.960953712463379
11 tensor(5.4059) tensor(10) 14.8333101272583
12 tensor(4.9408) tensor(10) 11.407886505126953
13 tensor(5.3353) tensor(10) 5.6867852210998535
14 tensor(5.8146) tensor(10) 12.572641372680664
15 tensor(5.2359) tensor(10) 12.889852523803711
16 tensor(5.7077) tensor(10) 0.9585597515106201
17 tensor(5.6412) tensor(10) 11.367023468017578
18 tensor(4.9677) tensor(10) 0.15255585312843323
19 tensor(4.5856) tensor(10) 3.586513757705688

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


20 tensor(5.6905) tensor(10) 4.996903419494629
21 tensor(5.8402) tensor(10) 0.022804751992225647
22 tensor(4.5891) tensor(10) 12.38820743560791
23 tensor(4.8313) tensor(10) 0.12145807594060898
24 tensor(4.9947) tensor(10) 1.173056721687317
25 tensor(5.2840) tensor(10) 0.0015290842857211828
26 tensor(4.4083) tensor(10) 0.010398893617093563
27 tensor(5.5587) tensor(10) 0.3188296854496002
28 tensor(4.4120) tensor(10) 1.2780697345733643
29 tensor(4.6617) tensor(10) 2.5201566219329834
30 tensor(5.1302) tensor(10) 4.43228006362915
31 tensor(5.5587) tensor(10) 13.170045852661133
32 tensor(4.7426) tensor(10) 0.1971459537744522
33 tensor(6.1737) tensor(10) 10.091483116149902
34 tensor(5.7331) tensor(10) 2.068005084991455
35 tensor(3.9739) tensor(10) 5.699672698974609
36 tensor(5.8104) tensor(10) 4.089468955993652
37 tensor(4.5667) tensor(10) 0.5116944313049316
38 tensor(5.1712) tensor(10) 21.038000106811523
39 tensor(5.9099) tensor(10) 20.733112335205078
40 tensor(5.8472) tensor(10) 7.607860088

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:113: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


101 tensor(5.9546) tensor(10) 0.4038214087486267
102 tensor(5.5847) tensor(10) 3.048060894012451
103 tensor(4.1754) tensor(10) 2.0131568908691406
104 tensor(5.0270) tensor(16) 0.5572551488876343
105 tensor(-3.1707) tensor(2) 2.291604518890381
106 tensor(-3.6091) tensor(1) 3.2183549404144287
107 tensor(-5.) tensor(0) 2.6895432472229004
108 tensor(-5.) tensor(0) 2.452373504638672
109 tensor(-5.) tensor(0) 2.2026257514953613
110 tensor(-5.) tensor(0) 1.952307105064392
111 tensor(-5.) tensor(0) 1.709784746170044
112 tensor(-5.) tensor(0) 1.480151653289795
113 tensor(-5.) tensor(0) 1.266649842262268
114 tensor(-5.) tensor(0) 1.0710943937301636
115 tensor(-5.) tensor(0) 0.8945058584213257
116 tensor(-5.) tensor(0) 0.7379368543624878
117 tensor(-5.) tensor(0) 0.601183295249939
118 tensor(-2.3168) tensor(2) 0.08202207088470459
119 tensor(-5.) tensor(0) 0.37972915172576904
120 tensor(4.9287) tensor(10) 8.66071605682373
121 tensor(4.9869) tensor(10) 3.874953269958496
122 tensor(4.6307) tensor(10

271 tensor(4.6346) tensor(10) 0.08412874490022659
272 tensor(4.5704) tensor(10) 0.0822637602686882
273 tensor(5.0453) tensor(10) 0.012512081302702427
274 tensor(4.7554) tensor(17) 0.3844802677631378
275 tensor(-0.6383) tensor(4) 1.3403135538101196
276 tensor(4.5755) tensor(10) 0.41464149951934814
277 tensor(5.9037) tensor(10) 0.045325666666030884
278 tensor(-1.8212) tensor(3) 0.04597816988825798
279 tensor(4.0690) tensor(19) 0.0015678118215873837
280 tensor(4.8981) tensor(10) 1.59372878074646
281 tensor(5.7638) tensor(10) 2.556335926055908
282 tensor(4.7064) tensor(10) 0.018015937879681587
283 tensor(6.1801) tensor(10) 0.040369272232055664
284 tensor(5.1123) tensor(10) 0.11548405140638351
285 tensor(5.8991) tensor(10) 0.8424040079116821
286 tensor(4.4658) tensor(16) 2.7500202655792236
287 tensor(4.7471) tensor(10) 0.16400392353534698
288 tensor(-1.7339) tensor(3) 0.05670730397105217
289 tensor(5.1562) tensor(16) 0.8904451727867126
290 tensor(5.5596) tensor(10) 0.001061791554093361
291 

435 tensor(5.9913) tensor(10) 0.09563253819942474
436 tensor(4.8531) tensor(10) 0.17688384652137756
437 tensor(4.6617) tensor(10) 0.013029471039772034
438 tensor(5.5525) tensor(10) 0.01865643821656704
439 tensor(4.7817) tensor(10) 0.012599173001945019
440 tensor(4.1531) tensor(10) 0.07377523928880692
441 tensor(4.7375) tensor(10) 0.30941998958587646
442 tensor(5.4473) tensor(10) 0.11498194187879562
443 tensor(5.1854) tensor(10) 0.03525251895189285
444 tensor(4.4865) tensor(16) 0.11008837819099426
445 tensor(-1.4018) tensor(4) 0.006719037890434265
446 tensor(3.8744) tensor(16) 0.027338402345776558
447 tensor(4.8184) tensor(10) 0.2610212564468384
448 tensor(4.5087) tensor(10) 0.11075768619775772
449 tensor(4.2399) tensor(10) 0.00042331291479058564
450 tensor(4.8807) tensor(10) 0.16704581677913666
451 tensor(6.1243) tensor(10) 0.0010733207454904914
452 tensor(5.8359) tensor(10) 0.05789598450064659
453 tensor(4.5530) tensor(10) 0.11437059193849564
454 tensor(5.4193) tensor(10) 0.0052584959

599 tensor(3.6160) tensor(16) 0.27897322177886963
600 tensor(5.6610) tensor(10) 0.08305368572473526
601 tensor(5.1900) tensor(10) 0.32420313358306885
602 tensor(5.6365) tensor(10) 0.0005751386051997542
603 tensor(5.4257) tensor(10) 0.002188869519159198
604 tensor(5.4034) tensor(10) 0.05761110410094261
605 tensor(5.2121) tensor(10) 0.0005901240510866046
606 tensor(4.2206) tensor(10) 0.04889822006225586
607 tensor(6.0572) tensor(10) 0.027271585538983345
608 tensor(4.5928) tensor(16) 0.21081200242042542
609 tensor(-0.6614) tensor(4) 0.21845147013664246
610 tensor(3.7225) tensor(16) 0.015009390190243721
611 tensor(-2.7744) tensor(2) 0.17164327204227448
612 tensor(3.0998) tensor(19) 0.0002959874109365046
613 tensor(-1.1850) tensor(3) 0.0017694763373583555
614 tensor(-1.0787) tensor(3) 0.07047167420387268
615 tensor(4.9736) tensor(10) 0.3459344804286957
616 tensor(5.0100) tensor(16) 0.1499868929386139
617 tensor(5.5089) tensor(10) 0.005420934408903122
618 tensor(5.7465) tensor(10) 0.02875462

764 tensor(-2.0670) tensor(3) 0.00033139335573650897
765 tensor(-1.7649) tensor(4) 0.01650017686188221
766 tensor(-3.0441) tensor(2) 0.001225072774104774
767 tensor(-0.8083) tensor(4) 0.013739057816565037
768 tensor(4.4663) tensor(10) 0.06292653828859329
769 tensor(5.1548) tensor(16) 0.21890924870967865
770 tensor(5.0730) tensor(10) 0.07959505170583725
771 tensor(6.0711) tensor(10) 0.00013052014401182532
772 tensor(5.9390) tensor(10) 0.04587551951408386
773 tensor(5.8111) tensor(10) 0.19542378187179565
774 tensor(5.6749) tensor(10) 0.009615601971745491
775 tensor(6.2582) tensor(10) 0.3283005356788635
776 tensor(-2.2218) tensor(2) 0.012621153146028519
777 tensor(2.5233) tensor(24) 0.013423094525933266
778 tensor(2.8800) tensor(21) 0.013115065172314644
779 tensor(2.5233) tensor(24) 0.10554678738117218
780 tensor(2.7864) tensor(19) 0.033469315618276596
781 tensor(2.5233) tensor(24) 0.026327088475227356
782 tensor(2.6867) tensor(23) 0.01847309246659279
783 tensor(-0.4802) tensor(4) 0.00105

929 tensor(4.8882) tensor(10) 0.44685837626457214
930 tensor(5.4102) tensor(10) 0.9564367532730103
931 tensor(5.7424) tensor(10) 0.6134220957756042
932 tensor(5.6221) tensor(10) 0.8009746074676514
933 tensor(5.3590) tensor(10) 0.16530300676822662
934 tensor(5.1600) tensor(16) 0.03706437349319458
935 tensor(5.5785) tensor(10) 2.7901044177269796e-06
936 tensor(-1.8010) tensor(3) 0.0006390060298144817
937 tensor(4.0458) tensor(17) 4.335986137390137
938 tensor(-1.2433) tensor(4) 0.014772075228393078
939 tensor(4.9267) tensor(10) 0.0029208322521299124
940 tensor(5.1143) tensor(10) 0.027745945379137993
941 tensor(5.7785) tensor(10) 0.35318487882614136
942 tensor(4.2237) tensor(10) 0.11634155362844467
943 tensor(3.9669) tensor(16) 1.341852068901062
944 tensor(4.8184) tensor(10) 0.02755691297352314
945 tensor(4.9485) tensor(10) 0.0017083939164876938
946 tensor(6.2182) tensor(10) 0.016862856224179268
947 tensor(4.4582) tensor(10) 0.01141073927283287
948 tensor(5.1511) tensor(10) 0.5996536016464

In [624]:
import torch.optim as optim
from torch.autograd import Variable
class g2apara:
    def __init__(self):
        self.rnn_hidden_dim=64
        self.n_agents=args.solDim
        self.attention_dim=16
        self.cuda=False
        self.n_actions=2
        self.hard=False
args3=g2apara()
g2a=G2ANet(1,args3)
def weights_init(mod):
    """设计初始化函数"""
    classname=mod.__class__.__name__
    # 返回传入的module类型
    print(classname)
    if classname.find('Conv')!= -1:    #这里的Conv和BatchNnorm是torc.nn里的形式
        mod.weight.data.normal_(-0.1,0.2)
    elif classname.find('BatchNorm')!= -1:
        mod.weight.data.normal_(1.0,0.2) #bn层里初始化γ，服从（1，0.02）的正态分布
        mod.bias.data.fill_(0)  #bn层里初始化β，默认为0
g2a.apply(weights_init)
optimizer = optim.SGD(g2a.parameters(), lr=1e-3, momentum=1e-1)
g2a.train()
hids=torch.from_numpy(np.random.random([args.solDim,64])).float()
rewards=[]
gamma=0.99
net = torch.nn.Sequential(
        torch.nn.Linear(args.solDim, 200),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(200, 100),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(100, 50),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(50, 1),
    )
optimizer1 = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss
def sigmoid(x):
    return (1/(1+torch.exp(-10*x)))

for t in range(1000):
    if t<100:
        tmp=[1]*args.card+[0]*(args.solDim-args.card)
        np.random.shuffle(tmp)
        res=torch.tensor(tmp)
    else:
        action=g2a.forward(torch.tensor([1.]*args.solDim).view(args.solDim,1),hids)
        tmp=(action-(np.array([i[1] for i in action])[np.argsort(np.array([i[1] for i in action]))[-(args.card+1)]]).item()).float()
        res=torch.cat(([sigmoid(tmp[i][1]).view(1) if sigmoid(tmp[i][1])!=0.5 else torch.tensor([0.])  for i in range(args.solDim)]),0)
        #等效netD.zero_grad()
        reward=hwithoutconstraints(res)-10*checkFea(res)#-constraint(torch.cat(tuple([i[1].view(1) for i in action]),0))/10000+((sum(([i[1] for i in action]))-args.card)**2)+(sum(([float(i[1]>0.5) for i in action]))-args.card)**2+1*entropy
        R = torch.zeros(1)
        R = gamma * R + reward
        loss=-reward
        #loss =-Variable(R)
        #loss=sum((sum(action)**2))/10#,torch.tensor(sum((1000*sum(action)**4)),requires_grad=True)
        #(hwithoutconstraints(torch.tensor([float(i[1]>0.5) for i in action]))-10*checkFea(torch.tensor([float(i[1]>0.5) for i in action])))
        loss.backward()#(retain_graph=True)  
        optimizer.step()   # update parameters
    prediction=net(torch.tensor(res).detach().float())
    loss1 = loss_func(prediction, hwithoutconstraints(torch.tensor(res).float()))
    print(t,hwithoutconstraints(torch.tensor(res).float())-10*checkFea(res),sum(res),loss1.item())
    optimizer1.zero_grad()   # clear gradients for next train
    loss1.backward()         # backpropagation, compute gradients
    optimizer1.step()        # apply gradients

Linear
GRUCell
GRU
Linear
Linear
Linear
Linear
Linear
G2ANet
0 tensor(5.7774) tensor(10) 42.15855407714844
1 tensor(4.6360) tensor(10) 41.209129333496094
2 tensor(5.1774) tensor(10) 33.51768493652344
3 tensor(5.5746) tensor(10) 28.379491806030273
4 tensor(5.7563) tensor(10) 21.682893753051758
5 tensor(6.3303) tensor(10) 9.951607704162598
6 tensor(5.1170) tensor(10) 1.2888208627700806
7 tensor(4.4335) tensor(10) 29.707040786743164
8 tensor(5.9060) tensor(10) 23.42815399169922
9 tensor(6.0730) tensor(10) 1.9457974433898926
10 tensor(5.2064) tensor(10) 0.7054248452186584
11 tensor(4.9036) tensor(10) 2.673002004623413
12 tensor(5.1585) tensor(10) 11.038993835449219
13 tensor(5.0285) tensor(10) 7.106042861938477
14 tensor(4.8142) tensor(10) 9.528114318847656
15 tensor(6.6432) tensor(10) 15.954068183898926
16 tensor(5.4230) tensor(10) 11.607589721679688
17 tensor(5.6633) tensor(10) 7.153092861175537
18 tensor(5.6612) tensor(10) 6.1270551681518555
19 tensor(5.4729) tensor(10) 3.34522342681884

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


23 tensor(5.6393) tensor(10) 3.052448034286499
24 tensor(5.8632) tensor(10) 8.36405086517334
25 tensor(4.8092) tensor(10) 0.02524232305586338
26 tensor(5.0386) tensor(10) 0.7223255634307861
27 tensor(5.1214) tensor(10) 0.042467180639505386
28 tensor(4.8125) tensor(10) 3.408743143081665
29 tensor(5.0169) tensor(10) 1.0830206871032715
30 tensor(5.1370) tensor(10) 0.4186464250087738
31 tensor(4.7107) tensor(10) 0.029691515490412712
32 tensor(5.5794) tensor(10) 5.71900749206543
33 tensor(4.9480) tensor(10) 4.424764156341553
34 tensor(5.5988) tensor(10) 0.03430671617388725
35 tensor(4.8514) tensor(10) 0.061232149600982666
36 tensor(5.2436) tensor(10) 0.601627767086029
37 tensor(5.1587) tensor(10) 10.324333190917969
38 tensor(6.0326) tensor(10) 0.3965427279472351
39 tensor(5.9351) tensor(10) 0.2763916850090027
40 tensor(4.9432) tensor(10) 1.1686679124832153
41 tensor(3.7121) tensor(10) 0.2260790318250656
42 tensor(5.4883) tensor(10) 6.048945903778076
43 tensor(5.1273) tensor(10) 2.9978499412

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:113: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


100 tensor(4.9309, grad_fn=<SubBackward0>) tensor(10.8987, grad_fn=<AddBackward0>) 0.01437354926019907
101 tensor(4.9319, grad_fn=<SubBackward0>) tensor(10.8907, grad_fn=<AddBackward0>) 0.08098292350769043
102 tensor(4.9341, grad_fn=<SubBackward0>) tensor(10.8743, grad_fn=<AddBackward0>) 0.1363697350025177
103 tensor(4.9378, grad_fn=<SubBackward0>) tensor(10.8504, grad_fn=<AddBackward0>) 0.1291397660970688
104 tensor(4.9434, grad_fn=<SubBackward0>) tensor(10.8211, grad_fn=<AddBackward0>) 0.07249346375465393
105 tensor(4.9513, grad_fn=<SubBackward0>) tensor(10.7897, grad_fn=<AddBackward0>) 0.017676418647170067
106 tensor(5.1206, grad_fn=<SubBackward0>) tensor(10.7613, grad_fn=<AddBackward0>) 0.00033422920387238264
107 tensor(5.1044, grad_fn=<SubBackward0>) tensor(10.6530, grad_fn=<AddBackward0>) 0.011105861514806747
108 tensor(5.1480, grad_fn=<SubBackward0>) tensor(10.7268, grad_fn=<AddBackward0>) 0.0856223851442337
109 tensor(5.3215, grad_fn=<SubBackward0>) tensor(10.7343, grad_fn=<Add

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [621]:
hwithoutconstraints(res)

tensor(0.)

In [625]:
torch.cat(([sigmoid(tmp[i][1]).view(1) if not 0.4<sigmoid(tmp[i][1])<0.6 else torch.tensor([0.])  for i in range(args.solDim)]),0)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [623]:
action

tensor([[5.7923e-10, 1.0000e+00],
        [1.5067e-09, 1.0000e+00],
        [8.6818e-09, 1.0000e+00],
        [1.5314e-09, 1.0000e+00],
        [4.8761e-09, 1.0000e+00],
        [1.2616e-08, 1.0000e+00],
        [6.0402e-10, 1.0000e+00],
        [3.3469e-09, 1.0000e+00],
        [1.2605e-09, 1.0000e+00],
        [1.9728e-09, 1.0000e+00],
        [6.7565e-10, 1.0000e+00],
        [2.6504e-10, 1.0000e+00],
        [2.5967e-09, 1.0000e+00],
        [1.0460e-09, 1.0000e+00],
        [1.0371e-09, 1.0000e+00],
        [3.2712e-09, 1.0000e+00],
        [1.8821e-09, 1.0000e+00],
        [1.4712e-09, 1.0000e+00],
        [2.0701e-09, 1.0000e+00],
        [9.6826e-10, 1.0000e+00],
        [1.4875e-09, 1.0000e+00],
        [6.9917e-09, 1.0000e+00],
        [3.5491e-09, 1.0000e+00],
        [6.6663e-10, 1.0000e+00]], grad_fn=<SoftmaxBackward>)